# Samples: Load Balancing

Sets up an APIM instance that demonstrates load balancing and circuit breaking across backends. 

⚙️ **Supported infrastructures**: apim-aca, afd-apim (with ACA)

⌚ **Expected *Run All* runtime (excl. infrastructure prerequisite): ~3 minutes**

## Objectives

1. Understand how backends can be configured to balance load in a prioritized, weighted manner.
1. Learn how circuit breakers aid with load balancing.
1. Configure how retries in API Management policies can result in more successful requests.

## Lab Components

This lab integrates into an existing Azure Container Apps architecture and sets up the following:

- One container app that serves multiple mock Web API endpoints returning 429 error codes. 
- Three separate backends are set up in APIM that each point to a different endpoint on this container app (e.g. /api/0, /api/1, etc.).
- Four separate backend pool with varying load balancer setups are configured using these three backends.

## Configuration

1. Decide which of the [Infrastructure Architectures](../../README.md#infrastructure-architectures) you wish to use.
    1. If the infrastructure _does not_ yet exist, navigate to the desired [infrastructure](../../infrastructure/) folder and follow its README.md.
    1. If the infrastructure _does_ exist, adjust the `user-defined parameters` in the _Initialize notebook variables_ below. Please ensure that all parameters match your infrastructure.

### Initialize notebook variables

Configures everything that's needed for deployment. 

[ADD ANY SPECIAL INSTRUCTIONS]

**Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.APIM_ACA
tags        = ['load-balancing']       # [ENTER DESCRIPTIVE TAG(S)]
api_prefix  = 'lb-'                    # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES

# 2) Service-defined parameters (please do not change these)
rg_name = utils.get_infra_rg_name(deployment, index)
supported_infrastructures = [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA]
utils.validate_infrastructure(deployment, supported_infrastructures)

# 3) Define the APIs and their operations and policies

# Policies
aca_backend_pool_prioritized_policy_xml                 = utils.read_policy_xml('./aca-backend-pool-prioritized.xml')
aca_backend_pool_prioritized_and_weighted_policy_xml    = utils.read_policy_xml('./aca-backend-pool-prioritized-and-weighted.xml')
aca_backend_pool_weighted_equal_policy_xml              = utils.read_policy_xml('./aca-backend-pool-weighted-50-50.xml')
aca_backend_pool_weighted_unequal_policy_xml            = utils.read_policy_xml('./aca-backend-pool-weighted-80-20.xml')

# Standard GET Operation
get = GET_APIOperation('This is a standard GET')

# ACA Backend Pools
apis: List[API] = [
    API(f'{api_prefix}prioritized-aca-pool', 'Prioritized backend pool', f'/{api_prefix}prioritized', 'This is the API for the prioritized backend pool.', policyXml = aca_backend_pool_prioritized_policy_xml, operations = [get], tags = tags),
    API(f'{api_prefix}prioritized-weighted-aca-pool', 'Prioritized & weighted backend pool', f'/{api_prefix}prioritized-weighted', 'This is the API for the prioritized & weighted backend pool.', policyXml = aca_backend_pool_prioritized_and_weighted_policy_xml, operations = [get], tags = tags),
    API(f'{api_prefix}weighted-equal-aca-pool', 'Weighted backend pool (equal)', f'/{api_prefix}weighted-equal', 'This is the API for the weighted (equal) backend pool.', policyXml = aca_backend_pool_weighted_equal_policy_xml, operations = [get], tags = tags),
    API(f'{api_prefix}weighted-unequal-aca-pool', 'Weighted backend pool (unequal)', f'/{api_prefix}weighted-unequal', 'This is the API for the weighted (unequal) backend pool.', policyXml = aca_backend_pool_weighted_unequal_policy_xml, operations = [get], tags = tags)
]

utils.print_ok('Notebook initialized')

### Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]}
}

# 2) Infrastructure must be in place before samples can be layered on top
if not utils.does_resource_group_exist(rg_name):
    utils.print_error(f'The specified infrastructure resource group and its resources must exist first. Please check that the user-defined parameters above are correctly referencing an existing infrastructure. If it does not yet exist, run the desired infrastructure in the /infra/ folder first.')
    raise SystemExit(1)

# 3) Run the deployment
output = utils.create_bicep_deployment_group(rg_name, rg_location, deployment, bicep_parameters)

# 4) Print a deployment summary, if successful; otherwise, exit with an error
if not output.success:
    raise SystemExit('Deployment failed')

if output.success and output.json_data:
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    app_insights_name = output.get('applicationInsightsName', 'Application Insights Name')

utils.print_ok('Deployment completed')

### Verify API Request Success

Assert that the deployment was successful by making simple calls to Azure Front Door or API Management.

In [ ]:
import time
import utils
from apimrequests import ApimRequests

def zzzs():
    sleep_in_s = 5
    utils.print_message(f'Waiting for {sleep_in_s} seconds for the backend timeouts to reset before starting the next set of calls', blank_above = True)
    time.sleep(sleep_in_s)  # Wait a bit before the next set of calls to allow for the backend timeouts to reset

# Preflight: Check if the infrastructure architecture deployment uses Azure Front Door. If so, assume that APIM is not directly accessible and use the Front Door URL instead.
utils.print_message('Checking if the infrastructure architecture deployment uses Azure Front Door.', blank_above = True)
afd_endpoint_url = utils.get_frontdoor_url(deployment, rg_name)

if afd_endpoint_url:
    reqs = ApimRequests(afd_endpoint_url)
    utils.print_message(f'Using Azure Front Door URL: {afd_endpoint_url}', blank_above = True)
else:
    reqs = ApimRequests(apim_gateway_url)
    utils.print_message(f'Using APIM Gateway URL: {apim_gateway_url}', blank_above = True)

# 1) Prioritized API calls
utils.print_message('1/4: Starting API calls for prioritized distribution (50/50)')
api_results_prioritized = reqs.multiGet('/lb-prioritized', runs = 15, msg = 'Calling prioritized APIs')
zzzs()

# 2) Weighted API calls
utils.print_message('2/4: Starting API calls for weighted distribution (50/50)', blank_above = True)
api_results_weighted_equal = reqs.multiGet('/lb-weighted-equal', runs = 15, msg = 'Calling weighted (equal) APIs')
zzzs()

# 3) Weighted API calls
utils.print_message('3/4: Starting API calls for weighted distribution (80/20)', blank_above = True)
api_results_weighted_unequal = reqs.multiGet('/lb-weighted-unequal', runs = 15, msg = 'Calling weighted (unequal) APIs')
zzzs()

# 4) Prioritized & weighted API calls
utils.print_message('4/4: Starting API calls for prioritized & weighted distribution', blank_above = True)
api_results_prioritized_and_weighted = reqs.multiGet('/lb-prioritized-weighted', runs = 20, msg = 'Calling prioritized & weighted APIs')
zzzs()

# 5) Prioritized & weighted API calls (500ms sleep)
utils.print_message('5/4: Starting API calls for prioritized & weighted distribution (500ms sleep)', blank_above = True)
api_results_prioritized_and_weighted_sleep = reqs.multiGet('/lb-prioritized-weighted', runs = 20, msg = 'Calling prioritized & weighted APIs', sleepMs = 500)

utils.print_ok('All done!')

### 🔍 Analyze Load Balancing results

The priority 1 backend will be used until TPM exhaustion sets in, then distribution will occur near equally across the two priority 2 backends with 50/50 weights.  

Please note that the first request of the lab can take a bit longer and should be discounted in terms of duration.

In [ ]:
import charts

charts.BarChart(
    api_results = api_results_prioritized,
    title = 'Prioritized Distribution',
    x_label = 'Run #',
    y_label = 'Response Time (ms)',
    fig_text = 'The chart shows a total of 15 requests across a prioritized backend pool with two backends.\n' \
        'Each backend, in sequence, was able to serve five requests for a total of ten requests until the pool became unhealthy (all backends were exhausted).\n' \
        'The average response time is calculated excluding statistical outliers above the 95th percentile (the first request usually takes longer).'
).plot()

charts.BarChart(
    api_results = api_results_weighted_equal,
    title = 'Weighted Distribution (50/50)',
    x_label = 'Run #',
    y_label = 'Response Time (ms)',
    fig_text = 'The chart shows a total of 15 requests across an equally-weighted backend pool with two backends.\n' \
        'Each backend, alternatingly, was able to serve five requests for a total of ten requests until the pool became unhealthy (all backends were exhausted).\n' \
        'The average response time is calculated excluding statistical outliers above the 95th percentile (the first request usually takes longer).'
).plot()

charts.BarChart(
    api_results = api_results_weighted_unequal,
    title = 'Weighted Distribution (80/20)',
    x_label = 'Run #',
    y_label = 'Response Time (ms)',
    fig_text = 'The chart shows a total of 15 requests across an unequally-weighted backend pool with two backends.\n' \
        'Each backend was able to serve requests for a total of ten requests until the pool became unhealthy (all backends were exhausted).\n' \
        'The average response time is calculated excluding statistical outliers above the 95th percentile (the first request usually takes longer).'
).plot()

charts.BarChart(
    api_results = api_results_prioritized_and_weighted,
    title = 'Prioritized & Weighted Distribution',
    x_label = 'Run #',
    y_label = 'Response Time (ms)',
    fig_text = 'The chart shows a total of 20 requests across a prioritized and equally-weighted backend pool with three backends.\n' \
        'The first backend is set up as the only priority 1 backend. It serves its five requests before the second and third backends - each part of\n' \
        'priority 2 and weight equally - commence taking requests.\n' \
        'The average response time is calculated excluding statistical outliers above the 95th percentile (the first request usually takes longer).'
).plot()

charts.BarChart(
    api_results = api_results_prioritized_and_weighted_sleep,
    title = 'Prioritized & Weighted Distribution (500ms sleep)',
    x_label = 'Run #',
    y_label = 'Response Time (ms)',
    fig_text = 'The chart shows a total of 20 requests across a prioritized and equally-weighted backend pool with three backends (same as previously).\n' \
        'The key difference to the previous chart is that each request is now followed by a 500ms sleep, which allows timed-out backends to recover.\n' \
        'The average response time is calculated excluding statistical outliers above the 95th percentile (the first request usually takes longer).'
).plot()
